In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
import scipy.stats
import os
import scipy.io as sio
import dnatools
from collections import Counter
%matplotlib inline
from pylab import *
# Plotting Params:
rc('mathtext', default='regular')
fsize=14

/net/gs/vol3/software/modules-sw-python/2.7.3/pandas/0.14.0/Linux/RHEL6/x86_64/lib/python2.7/site-packages/pandas/io/excel.py:626: UserWarning: Installed openpyxl is not supported at this time. Use >=1.6.1 and <2.0.0.
  .format(openpyxl_compat.start_ver, openpyxl_compat.stop_ver))


### Getting a mapping between intronic sequences and 3' UTR barcodes

This ipython notebook first gets all of the intronic sequence to 3'UTR barcode mappings. It then counts the number of spliced reads for each sequence at each position. Some important information:<br>
1. We only want to look at reads with the first 6 nt of the index read matching 'CACTGT'<br>
2. Read2 reads the 3'UTR barcode and then into the fixed sequence after<br>
NNNNNNNNNNNNNNNNNNNNGCAGGTAATG<br>
I only count reads that at least include caggtaat in the first 10nt. This is a little less stringent than requiring a perfect match.<br>
3. Read 1 actually starts 1nt past the exon-exon junction<br>
Plasmid reads will look like this:<br>
TGCTTGGNNNNNNNNNNNNNNNNNNNNNNNNNGGTCGACCCAGGTTCGTGNNNNNNNNNNNNNNNNNNNNNNNNNGAGGTATTCTTATCACCTTCGTGGCT<br>

In [2]:
resultsdir = '../results/N0_A5SS_Fastq_to_Splice_Reads/'
if not os.path.exists(resultsdir):
    os.makedirs(resultsdir)
figdir = '../figures/N0_A5SS_Fastq_to_Splice_Reads/'
if not os.path.exists(figdir):
    os.makedirs(figdir)
    
#Choose if you want to actually save the plots:
SAVEFIGS = True

In [3]:
f = {}
f[0] = open('../fastq/A5SS_dna_R1.fq','r')
f[1] = open('../fastq/A5SS_dna_R2.fq','r')
tagcount = {}
count = 0
tagqual = {}
header = {}
seq = {}
strand = {}
quality ={}
while True:
    for i in range(2):
        header[i] = f[i].readline()[:-1]
        seq[i] = f[i].readline()[:-1]
        strand[i] = f[i].readline()[:-1]
        quality[i] = f[i].readline()[:-1]
    tag = dnatools.reverse_complement(seq[1][:30])

    if len(quality[1])==0:
        break # End of File
    if (tag[:10].count('ATTACCTG')>0):
        qual = np.array([ord(i)-66 for i in quality[1][:20]])
        try:
            tagcount[tag] = tagcount[tag] + 1.
        except:
            tagcount[tag] = 1
            tagqual[tag] = qual
        else:
            tagqual[tag]+=qual
    if(count%1000000)==0:
        print count,len(tagcount),'|',
    count = count +1
f[0].close()
f[1].close()

tagqual = pd.Series(tagqual)
tagcount = pd.Series(tagcount)

0 1 | 1000000 347487 |


Get sequences for tags:
If a sequence for a tag is observed twice it is assumed to be the true mapping.  For tags, which have no
sequence found twice, the highest quality read sequence is chosen as the mapping

In [23]:
tagcount.max()

1810.0

### Filter 3' barcode tag sequences

I will only keep barcodes that have a minimum average pred of 21 or more. I will also only keep barcodes that were observed at least twice.

In [4]:
filter_thresh = 21
count_thresh = 2
DNA_index='CACTGT'

In [5]:
#Filter tags
# I summed the tag quality at every base, so to get the average, divide
# by the number of tag counts.
tag_qual_min = (tagqual/tagcount).apply(min)

tags_filtered = tag_qual_min[(tag_qual_min>=filter_thresh)&(tagcount>=count_thresh)]
print 'Number of Tags', len(tags_filtered)

Number of Tags 304288


### Create a map between 3' barcodes and Intronic Sequences

To get the mapping between intronic randomized sequences and the corresponding 3' barcodes, we will find the first sequence that occurs twice with the same barcode. This might not be the optimal solution, but it is efficient. For the barcodes that do not have an intronic sequence occuring together twice, we will take the sequence with the highest minimum base quality score.

In [6]:
tag_seqs = {}
mapped_tag2seq = {}
for tag in tags_filtered.index:
    tag_seqs[tag] = {}

In [7]:
f = {}
f[0] = open('../fastq/A5SS_dna_R1.fq','r')
f[1] = open('../fastq/A5SS_dna_R2.fq','r')
count = 0
header = {}
seq = {}
strand = {}
quality ={}
while True:
    for i in range(2):
        header[i] = f[i].readline()[:-1]
        seq[i] = f[i].readline()[:-1]
        strand[i] = f[i].readline()[:-1]
        quality[i] = f[i].readline()[:-1]
    if len(quality[1])==0:
        break # End of File
    tag = dnatools.reverse_complement(seq[1][:30])
    try:
        mapped_tag2seq[tag]
    except:
        try:
            tag_seqs[tag][seq[0]]
        except:
            try:
                tag_seqs[tag]
            except:
                pass
            else:
                tag_seqs[tag][seq[0]] = quality[1]
        else:
            mapped_tag2seq[tag] = seq[0]
            del tag_seqs[tag]

    count = count + 1
    if (count%1000000)==0:
        print count,len(mapped_tag2seq),'|',

f[0].close()
f[1].close()

# Map tags that did not have any sequences with two observations based
# on minimum base quality score:
for tag in tag_seqs:
    max_min_qual = 0
    for seq in tag_seqs[tag]:
        cur_min_qual = min([ord(s)-66 for s in tag_seqs[tag][seq]])
        if cur_min_qual>max_min_qual:
            mapped_tag2seq[tag] = seq
            max_min_qual = cur_min_qual
print len(mapped_tag2seq)

1000000 182443 | 302519


In [8]:
tag2seqs = pd.Series(mapped_tag2seq)

Remove sequences with . or N:

In [9]:
tag2seqs = tag2seqs[~(tag2seqs.str.contains('\.') | tag2seqs.str.contains('N'))]

Remove sequences with deletions or insertions:

In [10]:
tag2seqs = tag2seqs[tag2seqs.apply(lambda s:(s[32:49]=='GGTCGACCCAGGTTCGT')&(s[:7]=='TGCTTGG')&(s[75:100]=='GAGGTATTCTTATCACCTTCGTGGC') & (not ('N' in s)))]

So unfortunately I did not save the tag2seqs in alphabetical order. Subsequent analysis must be done with my ordering. The barcodes and intronic sequences are identical to the sequences produced here, just in a different order. There is no way to reproduce this ordering, because it came from saving a dictionary. Instead we will just load my file:

In [11]:
tag2seqs = pd.read_csv('../data/A5SS_Seqs.csv',index_col=0).Seq

### Counting the Spliced Reads at Each Position For Each Plasmid

This contains the full intronic sequence plus part of the second exon.

In [12]:
intronseq = 'TGCTTGGNNNNNNNNNNNNNNNNNNNNNNNNNGGTCGACCCAGGTTCGTGNNNNNNNNNNNNNNNNNNNNNNNNNGAGGTATTCTTATCACCTTCGTGGCTACAGAGTTTCCTTATTTGTCTCTGTTGCCGGCTTATATGGACAAGCATATCACAGCCATTTATCGGAGCGCCTCCGTACACGCTATTATCGGACGCCTCGCGAGATCAATACGTATACCAGCTGCCCTCGATACATGTCTTGGCATCGTTTGCTTCTCGAGTACTACCTGGTTCCTCTTCTTTCTTTCTCTTCTCTTTCAGGACGGCAGCGTGCAGCTCGCCGACCACTACCAGCAGAACACCCCCATCGGCGACGGCCCCGTGCTGCTGCCCGACAACCACTACCTGAGCTACCAGTCCGCCCTGAGC'

To map the position of the spliced exon-exon junction, I simple map the last 20 nt of the read. For example, if the read was spliced at the first SD, then the 81-101 nt of the read will map 81-101 nt into the second exon. If the read was spliced 44nt 3' of the first SD, the 81-101 nt of the read will map 37-57 nt into the second exon. Unspliced reads will have the 81-101 nt of the read mapping within the intron. However, requiring an exact match for these 20 nt is very stringent and we may lose reads. So I will allow one mismatch. To do this efficiently, I precompute a mapping between all potential 1nt errors and the position the read should map to.

In [13]:
#Make a map of all 1nt mismatches to actuall positions in intron
intronseq_map = {}
bases = ['A','T','C','G']
for b in xrange(len(intronseq)):
    seq = intronseq[b:b+19]
    intronseq_map[seq]=b
    for pos in range(19):
        for b1 in bases:
            mut_seq = seq[:pos]+b1+seq[pos+1:]
            intronseq_map[mut_seq] = b

This computes the position of the exon junction as described above. I generate the read count matrix of plasmids (rows) and exon junction positions (columns) by using scipy's sparse matrices.

In [14]:
tag_index = dict(zip(tag2seqs.index,range(len(tag2seqs))))

bases = ['A','T','C','G']
watsoncrick = {'N':'N','.':'.','C':'G','G':'C','A':'T','T':'A'}


f = {}
f[0] = open('../fastq/A5SS_rna_R1.fq','r')
f[1] = open('../fastq/A5SS_rna_R2.fq','r')
tag_list = []
ss_list = []
count=0
used_reads=0
header = {}
seq = {}
strand = {}
quality ={}
while True:
    for i in range(2):
        header[i] = f[i].readline()[:-1]
        seq[i] = f[i].readline()[:-1]
        strand[i] = f[i].readline()[:-1]
        quality[i] = f[i].readline()[:-1]
    tag = dnatools.reverse_complement(seq[1][:30])    
    found = False
    if len(quality[1])==0:
        break # End of File

    try:
        plasmid_num = tag_index[tag]
    except:
        pass

    else:
        found = True
    if found:
        last20 = str(seq[0][82:101])
        try:
            splice_pos = intronseq_map[last20]
        except:
            pass
        else:
            splice_pos = 385-splice_pos #get splice position
            if(splice_pos>=0)&(splice_pos<304):
                tag_list.append(tag_index[tag])
                ss_list.append(splice_pos)
                used_reads += 1

    if ((count%1000000)==0):
        print count,used_reads,'|',
    count += 1
f[0].close()
f[1].close()

0 0 | 1000000 638192 | 2000000 1259923 | 3000000 1864253 | 4000000 2465283 | 5000000 3093399 | 6000000 3696937 | 7000000 4278083 | 8000000 4887881 | 9000000 5512395 | 10000000 6115250 | 11000000 6694008 | 12000000 7324952 | 13000000 7943569 | 14000000 8543184 | 15000000 9135218 | 16000000 9764341 | 17000000 10369947 | 18000000 10949916 | 19000000 11507487 | 20000000 12134084 | 21000000 12734212 | 22000000 13309266 |


Make the sparse matrix:

In [15]:
splices = {'A5SS':scipy.sparse.csr_matrix((list(np.ones_like(ss_list))+[0],
                                           (tag_list+[len(tag2seqs)-1],ss_list+[303])))}

In [16]:
sio.savemat('../data/A5SS_Reads.mat',splices)

### Filter reads that do not have the canonical GT/GC splice donor

We know that splice donor have either GT or GC in the +1 to +2 positions (in the intron). Other "spliced" reads are probably noise. So we will enforce this condition:

In [17]:
GT_GC_matrix = np.zeros_like(np.array(splices['A5SS'].todense()))
intronic_sequences = tag2seqs.values
for i in range(len(tag2seqs)):
    cur_seq = 'G'+intronic_sequences[i]
    for j in range(len(cur_seq)):
        cur_dinuc = cur_seq[j:j+2]
        if (cur_dinuc=='GT') | (cur_dinuc=='GC'):
            GT_GC_matrix[i,j]=1
    # Allow no splicing:
    GT_GC_matrix[i,-1]=1
    if(i%10000)==0:
        print i,

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000


In [18]:
splices_GC_GT = scipy.sparse.csr_matrix((np.array(splices['A5SS'].todense())*GT_GC_matrix),
                                        dtype=np.float64)

### Save final matrix

Combine with the A3SS data into a single file:

In [20]:
Reads = sio.loadmat('../data/Reads.mat')
Reads['A5SS'] = splices_GC_GT
sio.savemat('../data/Reads.mat',Reads)